Copyright © 2021, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# HMEQ Dataset : Build and Import Trained Models into SAS Model Manager on SAS Viya 3.5

This notebook provides an example of how to build and train a Python model and then import the model into SAS Model Manager using the fleet maintenance data set. Lines of code that must be modified by the user, such as directory paths are noted with the comment "_Changes required by user._".

_**Note:** If you download only this notebook and not the rest of the repository, you must also download the hmeq.csv file from the data folder in the examples directory. These files are used when executing this notebook example._

Here are the steps shown in this notebook:

1. Import and review data and preprocess for model training.
2. Build, train, and access an H2O.ai generalized linear estimator model.
3. Serialize the model into pickle or MOJO files.
4. Write the metadata JSON files needed for importing into SAS Model Manager.
4. Write a score code Python file for model scoring.
5. Zip the model, JSON, and score code files into an archive file.
6. Import the ZIP archive file to SAS Model Manager via the Session object and relevant function call.

### Python Package Imports

In [1]:
# Dataframes for data manipulations
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# Mathematical calculations and array handling
import numpy as np

# Pathing support
from pathlib import Path

# Import H2O and check the version
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

# sasctl interface for importing models
import sasctl.pzmm as pzmm
from sasctl import Session

In [2]:
h2o.__version__

'3.22.1.1'

On SAS Viya 3.5, models created in H2O versions 3.24 and under are only compatible in the binary model format. For H2O versions 3.26+, models can be in the MOJO or binary model format. If using a binary model, the H2O version on the SAS Viya server must match the version of H2O used to create the model.

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,6 hours 0 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.1
H2O cluster version age:,"2 years, 9 months and 7 days !!!"
H2O cluster name:,H2O_from_python_sclind_i4ws71
H2O cluster total nodes:,1
H2O cluster free memory:,15.93 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"locked, healthy"


### Import and Review Data Set

In [4]:
hmeqData = h2o.import_file('data/hmeq.csv',sep= ',')
hmeqData.shape

Parse progress: |█████████████████████████████████████████████████████████| 100%


(5960, 13)

### Preprocess Data

In [5]:
hmeqData['BAD'] = hmeqData['BAD'].asfactor()

train, validation, test = hmeqData.split_frame(ratios=[.6, .2], seed=42)

y = 'BAD'
x = list(hmeqData.columns)
x.remove(y)

### Create, Train, and Assess Model

In [6]:
glmFit = H2OGeneralizedLinearEstimator(family='binomial', model_id='glmfit', lambda_search=True)
glmFit.train(x=x, y=y, training_frame=train, validation_frame=validation)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [7]:
# Check the model performance and print its accuracy
glmPerf = glmFit.model_performance(test)
print(glmPerf.accuracy())

[[0.551348008992684, 0.8486897717666948]]


#### Choice Point
The next two cells represent the saving process for binary and MOJO H2O models respectively. 

In [8]:
h2o.save_model(glmFit, path='data/hmeqModels/H2OBinaryGLM')

'C:\\Users\\sclind\\Documents\\Python Scripts\\GitHub\\sassoftware\\python-sasctl\\examples\\data\\hmeqModels\\H2OBinaryGLM\\glmfit'

In [9]:
#glmFit.save_mojo(path='data/hmeqModels/H2OMOJOGLM')

### Register Model in SAS Model Manager with pzmm
*Commented out lines are for MOJO models*

In [10]:
modelPrefix = 'glmFit'
zipFolder = Path.cwd() / 'data/hmeqModels/H2OBinaryGLM/'
pzmm.PickleModel.pickleTrainedModel(_, glmFit, modelPrefix, zipFolder, isH2OModel=True, isBinaryModel=True)
#pzmm.PickleModle.pickleTrainedModel(_, glmFit, modelPrefix, zipFolder, isH2OModel=True)

In [11]:
trainDF = train.as_data_frame()
J = pzmm.JSONFiles()

# Write input variable mapping to a json file
J.writeVarJSON(trainDF[x], isInput=True, jPath=zipFolder)

# Set output variables and assign an event threshold, then write output variable mapping
outputVar = pd.DataFrame(columns=['EM_EVENTPROBABILITY', 'EM_CLASSIFICATION'])
outputVar['EM_CLASSIFICATION'] = trainDF[y].astype('category').cat.categories.astype('str')
outputVar['EM_EVENTPROBABILITY'] = 0.5 # Event threshold
J.writeVarJSON(outputVar, isInput=False, jPath=zipFolder)

# Write model properties to a json file
J.writeModelPropertiesJSON(modelName=modelPrefix,
                            modelDesc='',
                            targetVariable=y,
                            modelType='',
                            modelPredictors=x,
                            targetEvent=1,
                            numTargetCategories=1,
                            eventProbVar='EM_EVENTPROBABILITY',
                            jPath=zipFolder,
                            modeler='sasdemo')

# Write model metadata to a json file
J.writeFileMetadataJSON(modelPrefix, jPath=zipFolder)

inputVar.json was successfully written and saved to c:\Users\sclind\Documents\Python Scripts\GitHub\sassoftware\python-sasctl\examples\data\hmeqModels\H2OBinaryGLM\inputVar.json
outputVar.json was successfully written and saved to c:\Users\sclind\Documents\Python Scripts\GitHub\sassoftware\python-sasctl\examples\data\hmeqModels\H2OBinaryGLM\outputVar.json
ModelProperties.json was successfully written and saved to c:\Users\sclind\Documents\Python Scripts\GitHub\sassoftware\python-sasctl\examples\data\hmeqModels\H2OBinaryGLM\ModelProperties.json
fileMetaData.json was successfully written and saved to c:\Users\sclind\Documents\Python Scripts\GitHub\sassoftware\python-sasctl\examples\data\hmeqModels\H2OBinaryGLM\fileMetaData.json


In [12]:
import getpass
username = getpass.getpass()
password = getpass.getpass()
host = 'sas.demo.com'
sess = Session(host, username, password, protocol='http')

In [13]:
pzmm.ImportModel.pzmmImportModel(zipFolder, modelPrefix, 'BinaryH2OExample', trainDF[x], trainDF[y], predictmethod=None, isH2OModel=True, force=True)
#pzmm.ImportModel.pzmmImportModel(zipFolder, modelPrefix, 'MOJOH2OExample', trainDF[x], trainDF[y], predictMethod='{}.predict({})', isH2OModel=True, force=True)

All model files were zipped to c:\Users\sclind\Documents\Python Scripts\GitHub\sassoftware\python-sasctl\examples\data\hmeqModels\H2OBinaryGLM.
Model was successfully imported into SAS Model Manager as glmFit with UUID: e34d30a4-66dd-4648-ad75-c6e92f0b01f1.
Model score code was written successfully to c:\Users\sclind\Documents\Python Scripts\GitHub\sassoftware\python-sasctl\examples\data\hmeqModels\H2OBinaryGLM\glmFitScore.py and uploaded to SAS Model Manager
